## Introdução
Case 3 - Avaliação Modelo NBO

##### O objetivo deste trabalho é: 

Contar uma história que mostre se a recomendação do carrossel faz ou não sentido com base no histórico de compra do cliente.

A idéia é aplicar princípios de validação da metodologia CRISP.

Se você fosse um coordenador de DS, você deixaria este modelo ser implementado em produção?

NBO -> "Next Best Offer"
No dataset atual, é feito uma sugestão de 5 produtos e também traz o score de cada sugestão que é a probabilidade do consumidor comprar aquele produto.

#### Resumo Aula

- Aula 1: Criação dos grupos e introdução aos temas
- Aula 2: Fazer as análises exploratórias básicas

## Anotações do Professor
- 12M faz sentido olhando para 6M?
- Valores errados? Nulos? Zero?
- Outliers
- Distribuição dos produtos vendidos faz sentido?

## Limpeza do Dataset

In [49]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [81]:
df_hist = pd.read_csv('./data/CASE_03_Compras_Hist.csv', sep=';')
df_carrossel = pd.read_csv('./data/CASE_03_NBO_Result.csv', sep=';')

In [82]:
df_hist.dtypes

Consumidor            int64
Produto               int64
Quantidade Acc 12M    int64
Quantidade Acc 6M     int64
dtype: object

In [83]:
df_carrossel['Score NBO 1'] = df_carrossel['Score NBO 1'].str.replace(',', '.').astype(float)
df_carrossel['Score NBO 2'] = df_carrossel['Score NBO 2'].str.replace(',', '.').astype(float)
df_carrossel['Score NBO 3'] = df_carrossel['Score NBO 3'].str.replace(',', '.').astype(float)
df_carrossel['Score NBO 4'] = df_carrossel['Score NBO 4'].str.replace(',', '.').astype(float)
df_carrossel['Score NBO 5'] = df_carrossel['Score NBO 5'].str.replace(',', '.').astype(float)

In [84]:
df_carrossel.dtypes

Consumidor       int64
NBO 1            int64
NBO 2            int64
NBO 3            int64
NBO 4            int64
NBO 5            int64
Score NBO 1    float64
Score NBO 2    float64
Score NBO 3    float64
Score NBO 4    float64
Score NBO 5    float64
dtype: object

In [35]:
#Valores Nulos
df_hist[df_hist.isna().any(axis=1)]

,Consumidor,Produto,Quantidade Acc 12M,Quantidade Acc 6M


In [50]:
df_carrossel[df_carrossel.isna().any(axis=1)]

,Consumidor,NBO 1,NBO 2,NBO 3,NBO 4,NBO 5,Score NBO 1,Score NBO 2,Score NBO 3,Score NBO 4,Score NBO 5


In [52]:
#Valores inconsistentes
df_hist[df_hist['Quantidade Acc 12M'] < df_hist['Quantidade Acc 6M']]

,Consumidor,Produto,Quantidade Acc 12M,Quantidade Acc 6M


## Análise Exploratória

In [18]:
#Analise Inicial - Histórico de Compras
df_hist.describe()

,Consumidor,Produto,Quantidade Acc 12M,Quantidade Acc 6M
count,1048488.000,1048488.000,1048488.000,1048488.000
mean,9362.000,40.493,2.500,0.625
std,5404.867,23.082,1.708,1.099
min,1.000,1.000,0.000,0.000
25%,4681.000,21.000,1.000,0.000
50%,9362.000,41.000,2.000,0.000
75%,14043.000,60.000,4.000,1.000
max,18723.000,80.000,5.000,5.000


In [20]:
df_hist.sample(n=5)

,Consumidor,Produto,Quantidade Acc 12M,Quantidade Acc 6M
99724,1781,23,4,2
45775,818,48,3,0
42535,760,25,5,0
247871,4427,33,4,2
865074,15448,31,3,2


In [32]:
#Analise Inicial - Histórico de Recomendações
df_carrossel.describe()

,Consumidor,NBO 1,NBO 2,NBO 3,NBO 4,NBO 5,Score NBO 1,Score NBO 2,Score NBO 3,Score NBO 4,Score NBO 5
count,18723.000,18723.000,18723.000,18723.000,18723.000,18723.000,18723.000,18723.000,18723.000,18723.000,18723.000
mean,9362.000,10.574,27.017,42.057,58.506,73.476,0.744,0.744,0.745,0.746,0.745
std,5405.009,5.778,3.741,4.918,4.625,4.025,0.139,0.139,0.138,0.138,0.139
min,1.000,1.000,21.000,34.000,51.000,67.000,0.510,0.510,0.510,0.510,0.510
25%,4681.500,6.000,24.000,38.000,55.000,70.000,0.620,0.620,0.630,0.630,0.620
50%,9362.000,11.000,27.000,42.000,58.000,73.000,0.740,0.740,0.750,0.750,0.740
75%,14042.500,16.000,30.000,46.000,63.000,77.000,0.870,0.870,0.860,0.870,0.870
max,18723.000,20.000,33.000,50.000,66.000,80.000,0.980,0.980,0.980,0.980,0.980


In [33]:
df_carrossel.sample(n=5)

,Consumidor,NBO 1,NBO 2,NBO 3,NBO 4,NBO 5,Score NBO 1,Score NBO 2,Score NBO 3,Score NBO 4,Score NBO 5
5433,5434,13,24,45,59,78,0.510,0.960,0.690,0.530,0.730
11973,11974,10,26,45,53,77,0.740,0.850,0.770,0.570,0.680
18257,18258,6,30,38,56,68,0.580,0.970,0.750,0.830,0.610
8618,8619,8,22,43,60,80,0.580,0.740,0.870,0.730,0.560
4673,4674,19,24,41,61,74,0.960,0.880,0.870,0.970,0.630


In [14]:
#TOP 5 Produtos vendidos na base histórica
df_hist_top5 = df_hist['Produto'].value_counts().sort_values(ascending=False).head(5)
df_hist_top5

56    13444
19    13387
3     13360
14    13291
76    13284
Name: Produto, dtype: int64

In [42]:
#LOWEST 5 Produtos vendidos na base histórica
df_hist_lowest5 = df_hist['Produto'].value_counts().sort_values(ascending=True).head(5)
df_hist_lowest5

13    12845
47    12847
12    12873
26    12885
30    12907
Name: Produto, dtype: int64

In [47]:
#TOP 5 Produtos Mais Sugeridos no carrossel
a = df_carossel['NBO 1']
b = df_carossel['NBO 2']
c = df_carossel['NBO 3']
d = df_carossel['NBO 4']
e = df_carossel['NBO 5']

df_carrossel_top5 = pd.concat([a, b, c, d, e])
df_carrossel_top5 = df_carrossel_top5.value_counts().sort_values(ascending=False).head(5)
df_carrossel_top5

31    1476
27    1471
22    1463
32    1463
24    1460
dtype: int64

In [48]:
#LOWEST 5 Produtos Menos Sugeridos no carrossel
df_carrossel_lowest5 = pd.concat([a, b, c, d, e])
df_carrossel_lowest5 = df_carrossel_lowest5.value_counts().sort_values(ascending=True).head(5)
df_carrossel_lowest5

14    875
3     891
8     899
7     904
2     907
dtype: int64

In [59]:
#TOP 5 Consumidores (com base no numero de compras na base histórica) 6M
df_hist_top5con6M = df_hist[df_hist['Quantidade Acc 6M'] > 0]
df_hist_top5con6M = df_hist_top5con6M['Consumidor'].value_counts().sort_values(ascending=False).head(5)
df_hist_top5con6M

# Quantidade Acc 12M 	Quantidade Acc 6M

1008     33
11142    31
12681    31
11653    31
4265     31
Name: Consumidor, dtype: int64

In [60]:
#TOP 5 Consumidores (com base no numero de compras na base histórica) 12M
df_hist_top5con12M = df_hist[df_hist['Quantidade Acc 12M'] > 0]
df_hist_top5con12M = df_hist_top5con12M['Consumidor'].value_counts().sort_values(ascending=False).head(5)
df_hist_top5con12M

9406     56
5686     55
7507     55
15623    55
13090    55
Name: Consumidor, dtype: int64

In [62]:
#Trazer o NBO do TOP 5 Consumidores 6M
df_hist_top5con6M = pd.merge(df_hist_top5con6M, df_carrossel, how='inner', on=["Consumidor", "Consumidor"])
df_hist_top5con6M

,Consumidor,NBO 1,NBO 2,NBO 3,NBO 4,NBO 5,Score NBO 1,Score NBO 2,Score NBO 3,Score NBO 4,Score NBO 5
0,33,10,22,48,55,75,0.960,0.670,0.840,0.880,0.560
1,31,11,23,38,63,75,0.510,0.510,0.880,0.750,0.640
2,31,11,23,38,63,75,0.510,0.510,0.880,0.750,0.640
3,31,11,23,38,63,75,0.510,0.510,0.880,0.750,0.640
4,31,11,23,38,63,75,0.510,0.510,0.880,0.750,0.640


In [63]:
#Trazer o NBO do TOP 5 Consumidores 12M
df_hist_top5con12M = pd.merge(df_hist_top5con12M, df_carrossel, how='inner', on=["Consumidor", "Consumidor"])
df_hist_top5con12M

,Consumidor,NBO 1,NBO 2,NBO 3,NBO 4,NBO 5,Score NBO 1,Score NBO 2,Score NBO 3,Score NBO 4,Score NBO 5
0,56,9,31,45,54,77,0.530,0.650,0.870,0.530,0.930
1,55,4,28,34,55,74,0.540,0.630,0.900,0.950,0.540
2,55,4,28,34,55,74,0.540,0.630,0.900,0.950,0.540
3,55,4,28,34,55,74,0.540,0.630,0.900,0.950,0.540
4,55,4,28,34,55,74,0.540,0.630,0.900,0.950,0.540


#### REMARK: Qual a lógica por traz da recomendação dos 5 produtos do carrossel?
Cada NBO [1 ao 5] pode sugerir apenas um range de produtos como é demonstrado a seguir

=> Isto é ruim pois caso o cliente tenha um historico de compras somente para produtos da faixa (67 ao 80), os outros NBOs ficariam mal utilizados

In [68]:
df_carrossel['NBO 1'].describe()   # Produto 1  ao 20

count   18723.000
mean       10.574
std         5.778
min         1.000
25%         6.000
50%        11.000
75%        16.000
max        20.000
Name: NBO 1, dtype: float64

In [69]:
df_carrossel['NBO 2'].describe()    # Produto 21 ao 33

count   18723.000
mean       27.017
std         3.741
min        21.000
25%        24.000
50%        27.000
75%        30.000
max        33.000
Name: NBO 2, dtype: float64

In [70]:
df_carrossel['NBO 3'].describe()    # Produto 34 ao 50

count   18723.000
mean       42.057
std         4.918
min        34.000
25%        38.000
50%        42.000
75%        46.000
max        50.000
Name: NBO 3, dtype: float64

In [71]:
df_carrossel['NBO 4'].describe()    # Produto 51 ao 66

count   18723.000
mean       58.506
std         4.625
min        51.000
25%        55.000
50%        58.000
75%        63.000
max        66.000
Name: NBO 4, dtype: float64

In [72]:
df_carrossel['NBO 5'].describe()    # Produto 67 ao 80

count   18723.000
mean       73.476
std         4.025
min        67.000
25%        70.000
50%        73.000
75%        77.000
max        80.000
Name: NBO 5, dtype: float64

#### REMARK: Quantas variações de recomendações foram feitas?


In [96]:
def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]


df_carrossel['ConjuntoID'] = 99*(100**5) + (df_carrossel['NBO 1']*(100**4)) + (df_carrossel['NBO 2']*(100**3)) + (df_carrossel['NBO 3']*(100**2)) + (df_carrossel['NBO 4']*(100**1)) + (df_carrossel['NBO 5']*(100**0))
df_carrossel['ConjuntoID'].value_counts().sort_values(ascending=False)


990425376570    3
991322466167    2
991628375475    2
990122356372    2
991729416078    2
               ..
990421356674    1
991031456479    1
991523355668    1
990423496072    1
991127425669    1
Name: ConjuntoID, Length: 18562, dtype: int64